# Problems With Using the GLM to Model fMRI Data
Although we have now seen one way of using the GLM to model fMRI data, there are several problems with the approach described thus far. These include issues with the shape and timing of the BOLD response, the correlation structure of time-series data, the presence of low-frequency noise and the arbitrary scaling of BOLD units. Much of the early research into the mass-univariate framework was concerned with identifying and providing solutions to these problems. This can be seen in a series of papers from Friston and colleagues called [Analysis of the fMRI time-series](https://onlinelibrary.wiley.com/doi/abs/10.1002/hbm.460010207), [Analysis of the fMRI time-series - revisited](https://pubmed.ncbi.nlm.nih.gov/9343589/), and [Analysis of the fMRI time-series - revisited -- again](https://pubmed.ncbi.nlm.nih.gov/9343600/). These papers demonstrate how it took a number of years to identify and solve all the problems with applying the GLM to fMRI data. Even to this day, some of the "solutions" used by SPM are contentious. As such, it is important to understand both the problems and their purported solutions in order to understand some of the limitations of this analysis approach.

## Issue 1: The Shape and Timing of the BOLD Response
The first problem with the approach described in the previous section is the *shape* and *timing* of the predictor variables. So far, we have used a dummy variable to indicate the onset and offset of the experimental conditions. This implies an instantaneous change in signal at the start of each experimental condition, as well as an instantaneous change in signal at the end of the experimental condition. However, as we know, the BOLD response is far from instantaneous. In {numref}`hrf-fig` we can see a typical BOLD response to a stimulus at time 0. 

```{figure} images/hrf.png
---
width: 500px
name: hrf-fig
---
Illustration of a typical BOLD response to a single stimulus at time 0.
```

Notice how it takes around 6 seconds for the signal to reach its peak and around 20 seconds to come back to baseline. As such, if we use a dummy variable built from the onset times of the stimuli, we run the risk of our model not fitting the data simply because the peak BOLD response is offset by around 6 seconds. This lack of fit will lead to larger errors and a larger variance estimate. This will directly impact the magnitude of the standard errors, leading to a less sensitive model. This issue is illustrated in {numref}`bad-fit-fig`.

```{figure} images/bad-fit.png
---
width: 600px
name: bad-fit-fig
---
Illustration of how a dummy variable leads to poor correspondance with the BOLD signal given the delayed response to the stimuli.
```

One solution to this problem would be to offset the dummy variable in time to accommodate the delayed response. This would help improve the fit, but we would still not be capturing the *shape* of the actual response. In order to do so, we use the process of *convolution*, which we discussed earlier on the course. By convolving the dummy variable with a model of the hemodynamic response, we can create a much more realistic prediction of what the signal should look like. This is illustrated in {numref}`hrf-conv-fig`.

```{figure} images/hrf-convolve.png
---
width: 800px
name: hrf-conv-fig
---
Demonstration of how convolution of a hemodynamic response model (*left*) with a dummy variable (*middle*) can produce a prediction for the shape of the BOLD signal in response to the experimental manipulation (*right*).
```

This model of the hemodynamic response is known as the *hemodynamic response function* (HRF), and was derived by several authors in the mid-90s, using de-convolution methods. The specific version shown in {numref}`hrf-conv-fig` is referred to as the *canonical* HRF by SPM, and was created by combining two gamma distributions. As such, this is sometimes termed the *double-gamma* HRF. We will discuss more about the justification for the HRF and the convolution operator in the *Experimental Design & Optimisation* module. For now, just consider the comparison in {numref}`conv-error-fig` of a model without convolution (*left*) and with convolution (*right*). As should hopefully be clear, the model *with* convolution provides a much better fit to the fMRI time series. 

```{figure} images/convolution-error.png
---
width: 800px
name: conv-error-fig
---
Comparison between a model without convolution (*left*) and with convolution (*right*).
```

## Issue 2: Autocorrelation
The second problem with the approach described in the previous section is that it does not acknowledge that we are working with *time-series* data. A particularly important element of the theory behind the GLM is that this method is only correct if the data in $\mathbf{y}$ are *independent*. In other words, that there is no *correlation* between the data points. This matters because it impacts both the nature of the estimates and the standard errors. The presence of correlation thus influences the magnitude of the test statistics and the accuracy of the calculated *p*-values. So far, we have used the GLM as if the time-series contains no correlation. Unfortunately, time-series data have a very specific correlation structure known as *autocorrelation*. This means that values close in time are more correlated than values far away in time. Because of this, we need some way to take this correlation structure into account.

In order to accommodate the correlation in the time series, SPM performs an initial model fit at every voxel and then uses the errors to estimate the correlation structure in the data. This estimation is performed using an autoregressive model of order 1, usually shortened to AR(1), which you can read more about below.

```{admonition} Advanced: Why is an AR(1) Model?
:class: dropdown
An AR(1) model for the errors at time $t$ is

$$
\begin{align}
\epsilon_{t} &= \rho \epsilon_{t-1} + \tau_{t} \\
\tau_{t} &\sim \mathcal{N}(0,\sigma)
\end{align}
$$

Here, $\rho$ is the correlation and $\epsilon_{t-1}$ is the error one step back in time. The main consequence of this structure is that when the covariance (correlation) of two errors is calculated, you get

$$
\text{Cov}\left(\epsilon_{t},\epsilon_{t+n}\right) = \frac{\sigma^{2}}{1-\rho^{2}}\rho^{|n|}
$$

Raising the correlation to the power of $n$ means that it gets exponentially *weaker* the further apart in time the data points are. This therefore captures a simple version of autocorrelation that can be estimated efficiently by SPM using maximum likelihood methods. Of note, however, is that SPM does not estimate this correlation structure uniquely at each voxel. Instead, a pool of voxels is used to estimate a *single* correlation structure that is assumed to be the same everywhere in the brain. This is done in the name of computational efficiency, but will never be true in reality. This is a distinct weakness of the SPM implementation that is not found in either FSL or AFNI.
```

Once SPM has estimated the AR(1) model, it uses the estimated correlation structure to create a *whitening matrix* ($\mathbf{W}$) that can be used to remove the correlation from the data. SPM does this by pre-multiplying the data and design. The result is data that is (in theory) uncorrelated and thus is more suitable for use in the GLM.

$$
\mathbf{WY} = \mathbf{WX}\boldsymbol{\beta} + \mathbf{W}\boldsymbol{\epsilon}
$$

```{admonition} Whitening
:class: tip
The term *whitening* comes from the perspective of signal processing, where removal of the correlation makes the errors akin to *white noise*. From a practical perspective, after the whitening procedure the data and the design matrix will be different. This is because removing the correlation breaks the connection between the data and the predictor variables because that the shape of the BOLD response will change. In order to make sure our predictor variables are still accurate, they must also be adjusted by the whitening matrix. This is the reason why you will see the design matrix change colour in SPM during the course of the statistical modelling.
```

## Issue 3: Low-frequency Noise
The third problem with the approach described in the previous section is that the fMRI time series is often contaminated by low-frequency noise. This is also known as *signal drift* and is illustrated in {numref}`drift-fig`. Notice how this time series shows a steady increase in magnitude over time.

```{figure} images/drift.png
---
width: 800px
name: drift-fig
---
Illustration of signal drift caused by low-frequency noise in the fMRI time series.
```

The problem with signal drift is that it will bias the parameter estimates. For instance, we may get the impression that the signal change is larger in one experimental condition simply because that condition was repeated closer to the end of the experiment where the signal was *larger*. Drift can also be seen as a *decrease* in signal over time, which would have the opposite effect in terms of biasing the estimates. Unfortunately, the cause of drift relates to the way the scanner opperates (e.g. heating over time) and thus is difficult to avoid. As such, we have to find a way of removing the drift from the data.

```{admonition} Dynamic drift correction
:class: tip
Modern scanning sequences often have facilities to correct for signal drift during the data acquisition. These approaches are termed “Dynamic Stabilization” on Philips scanners, “Real Time Field Adjustment” on GE scanners, and “Frequency Stabilization” on Siemens scanners. In all cases, these methods correct for drift by recalibrating the scanner after each volume is collected, rather than a single calibration step at the beginning of the sequence. As such, this a feature that is worth checking for when the scan is being setup as it may diminish or remove the need for drift correction during the analysis. 
```

In order to remove drift, SPM passes each time series through a *high-pass filter*. This removes *low-frequency* information from the data (it passes *over* high-frequencies). The way this is implemented is by using a *discrete cosine transform* (DCT) basis set. This is essentially a series of cosines increasing in frequency up to the desired filter cutoff, as shown in {numref}`dct-fig`. The DCT basis set can be added as extra columns in the design matrix. This works in a similar fashion to the Fourier transform, in the sense that a linear combination of periodic functions can be used to represent any signal. In this case, the cosines will act together to remove any frequency *below* the cutoff from the data. As mentioned earlier, this relies on the fact that the parameter estimates from the GLM are interpreted as effects *after* all other variables are taken into account. As such, adding the DCT basis set means that all the parameters associated with the experiment will be interpreted as effects *after* the low-frequency information is removed.

```{figure} images/dct.png
---
width: 800px
name: dct-fig
---
Example of a DCT basis set.
```

In order to not clutter up the design matrix, recent iterations of SPM have taken a slightly different approach to implementing this filter. Instead of adding the cosines to the design matrix, SPM uses the cosines to form a *filtering matrix* ($\mathbf{S}$) which can be used to pre-multiply the data and design. This works in the same fashion as *whitening* 

$$
\mathbf{SY} = \mathbf{SX}\boldsymbol{\beta} + \mathbf{S}\boldsymbol{\epsilon}
$$

and produces *identical* results to adding the cosines to the design matrix. 

From a practical perspective, the only element of filtering we need to concern ourselves with is what cutoff to use. By default, SPM chooses 128 seconds, which is equivalent to $\frac{1}{128} = 0.008\text{Hz}$. As such, any frequencies *below* 0.008Hz will be removed from the data. Practically, this means we should design our experiments so that any periodic changes of interest (such as the on-off pattern of the experimental conditions) do not occur slower than every 2 minutes. If they do, we run the risk of removing experimental signal with the filter. Of course, we can change the filter, but this runs the risk of allowing more low-frequency noise to remain in the data. As we will see, there are facilities within SPM that can be used to find a suitable filter value, but this is an element that should be considered when designing the experiment. As an aside, this filtering is another reason why you will see the design matrix change colour during the course of the analysis.

## Issue 4: Image Scaling
The final problem with the approach described in the previous section is that the BOLD signal is measured on an arbitrary scale that can differ from subject-to-subject and scanner-to-scanner. The reason this is an issue is because the GLM parameters are on the same scale as the data. As we know from the previous example, the estimate of $\beta_{1}$ will indicate the change in BOLD signal for a unit change in the first predictor variable. Unfortunately, the magnitude of this effect cannot be meaningfully compared with that of another subjects, because the depends upon how the signal is scaled. To get around this, SPM will rescale the data such that the mean over all voxels and all volumes is equal to 100. This is known as *grand mean scaling* and is performed automatically in the background. The reason why we should be aware of this step is because we need to be careful about *interpretation*. The temptation might be to think that this scaling renders the parameters on some sort of standardised and interpretable scale, such as *percentage signal change*. However, this is not correct as the procedure to convert the parameters to percentage signal change is much more involved (see [Pernet, 2014](https://www.frontiersin.org/articles/10.3389/fnins.2014.00001/full)). So note that SPM automatically scales the data, but remember this does not result in easy to interpret units for the effects.

## The Adapted GLM
We have now seen that, in order to make the GLM suitable for fMRI data, it is necessary to convolve dummy variables with a HRF, estimate and remove the autocorrelation in the time series, filter the data to remove low-frequency noise and rescale the data to common units across subjects. All these steps form an *adapted* version of the GLM, as illustrated in {numref}`adapt-glm1-fig`

```{figure} images/adapted-glm-1.png
---
width: 800px
name: adapt-glm1-fig
---
The adapted fMRI GLM before parameter estimation.
```

where the $\star$ symbol indicates the *scaled*, *whitened* and *filtered* versions of the data, design matrix and errors. Once the parameters have been estimated, they can be used to multiply each column to scale the prediction, as shown in {numref}`adapt-glm2-fig`. Notice how the constant takes on the baseline level of signal (400 in this example) and that the conditions are simply changes relative to this baseline.

```{figure} images/adapted-glm-2.png
---
width: 800px
name: adapt-glm2-fig
---
The adapted fMRI GLM after the parameter estimates are used to scale the design matrix columns.
```

Adding all the scaled columns together provides the final model prediction, as shown in {numref}`adapt-glm3-fig`.

```{figure} images/adapted-glm-3.png
---
width: 500px
name: adapt-glm3-fig
---
The adapted fMRI GLM after the scaled design matrix columns are summed, producing the final prediction of the BOLD signal.
```
